In [1]:
!pip install tiktoken

  Using cached regex-2024.5.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 952.0 kB/s eta 0:00:0000:0100:01
Using cached regex-2024.5.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (775 kB)

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from openai import OpenAI
from dotenv import load_dotenv,find_dotenv
import instructor
import os
import min_search
import json
from pydantic import BaseModel, Field
import tiktoken
from tqdm import tqdm
import pandas as pd

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in tqdm(documents_raw):
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
    
load_dotenv(find_dotenv())

client = instructor.patch(OpenAI(), mode=instructor.Mode.TOOLS)

100%|██████████| 3/3 [00:00<00:00, 10255.02it/s]


In [6]:
pd.DataFrame(documents).head()

,text,section,question,course
0,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,data-engineering-zoomcamp
1,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,data-engineering-zoomcamp
2,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,data-engineering-zoomcamp
3,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,data-engineering-zoomcamp
4,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,data-engineering-zoomcamp


In [30]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
print(json.dumps(documents[0], indent=2))

{
  "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
  "section": "General course-related questions",
  "question": "Course - When will the course start?",
  "course": "data-engineering-zoomcamp"
}


In [31]:
for i, doc in enumerate(tqdm(documents)):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(f"Error indexing document {i}: {e}")
        print(f"Problematic document: {json.dumps(doc, indent=2)}")
        break

100%|██████████| 948/948 [00:06<00:00, 137.64it/s]


## Question 1: 

### Answer:

/: 42f05b9372a9a4a470db3b52817899b99a76ee73

## Question 2:

### Answer: 

index

# Question 3

### Answer: 84...

In [32]:
def elastic_search(query:str):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "machine-learning-zoomcamp"
                #     }
                # }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    return response['hits']['hits']



In [33]:
query = "How do I execute a command in a running docker container?"

In [34]:
elastic_search(query)

[{'_index': 'course-questions',
  '_id': 'zgk0b5ABYTodIUsMQGs6',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '3Ak0b5ABYTodIUsMMmke',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\n

#### Question 4:

answer: How do I copy files from a different folder into docker container’s working directory?

In [35]:
def elastic_search(query:str):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    return response['hits']['hits']



In [37]:
search_query = "How do I execute a command in a running docker container?"
answer_es = elastic_search(search_query)

In [40]:
context = ""

for retrieval in answer_es:
    source = retrieval['_source']
    context_template = """
Q: {question}
A: {text}
""".strip()
    context += context_template.format(question=source['question'], text=source['text'])




In [41]:
# Question 5

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context).strip()
len(prompt)


1458

In [42]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [43]:
# question 6

In [45]:
len(encoding.encode(prompt))

321

In [46]:
a = encoding.encode(prompt)
b = encoding.decode(a)
print(a)
print(b)

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 8, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 5150,